---
<a name = Section1></a>
# **1. Introduction**
---

**PROJECT DESCRIPTION:**
============================

See https://projects.insaid.co/capstone2/index.php


---
<a name = Section2></a>
# **2. Installing and importing libraries**
---



<a name = Section21></a>
### **2.1 Installing Libraries**

1.   Restart Runtime (in Colab it is menu -> Runtime -> Restart Runtime)
2.   Run the below steps


In [1]:
#!pip install -q datascience                   # Package that is required by pandas profiling
#!pip install -q pandas-profiling              # Library to generate basic statistics about data

In [2]:
#!pip install -q --upgrade pandas-profiling

In [3]:
#!pip install mysql-connector-python ## DB Connection ##

3.   Restart Runtime again 
4.   Run step 2 again
5.   Run the code below (to install the other libraries)

In [4]:
#------------------------------------------------------------------------------------------------------------------------------
import pandas as pd                                                 # Importing for panel data analysis
from pandas_profiling import ProfileReport                          # Import Pandas Profiling (To generate Univariate Analysis)
pd.set_option('display.max_columns', None)                          # Unfolding hidden features if the cardinality is high
pd.set_option('display.max_rows', None)                             # Unfolding hidden data points if the cardinality is high
pd.set_option('mode.chained_assignment', None)                      # Removing restriction over chained assignments operations
#-------------------------------------------------------------------------------------------------------------------------------
import numpy as np                                                  # Importing package numpys (For Numerical Python)
from scipy.stats import randint as sp_randint                       # For initializing random integer values
#-------------------------------------------------------------------------------------------------------------------------------
import matplotlib.pyplot as plt                                     # Importing pyplot interface using matplotlib
import seaborn as sns                                               # Importin seaborm library for interactive visualization
%matplotlib inline
#-------------------------------------------------------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler                    # To scaled data with mean 0 and variance 1
from sklearn.model_selection import train_test_split                # To split the data in training and testing part
from sklearn.model_selection import RandomizedSearchCV              # To find best hyperparamter setting for the algorithm
from sklearn.ensemble import RandomForestClassifier                 # To implement random forest classifier
from sklearn.tree import DecisionTreeClassifier                     # To implement decision tree classifier
from sklearn.metrics import classification_report                   # To generate classification report
from sklearn.metrics import plot_confusion_matrix                   # To plot confusion matrix
import pydotplus                                                    # To generate pydot file
from IPython.display import Image                                   # To generate image using pydot file
#-------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once

---
<a name = Section3></a>
# **3. Loading Train and Test data**
---



<a name = Section31></a>
### **3.1 Loading Training data**

In [18]:
#------ DB Connection ------
import mysql.connector
mydb = mysql.connector.connect(
 user='student', password='student',
 host='cpanel.insaid.co',
 database='Capstone2')

mycursor = mydb.cursor()

In [7]:
# --- Function get table data as a data frame ---
def getTableDataAsDataFrame(dbCursor, tableName):

  ## Get the Columns  of the table
  dbCursor.execute("show columns from " + tableName)
  cols_result = dbCursor.fetchall()
  cols_df = pd.DataFrame(cols_result)
  cols = cols_df.iloc[:, 0]

  ## Then, get the data stored into a DataFrame
  dbCursor.execute("select * from " + tableName)
  table_data_result = dbCursor.fetchall()
  table_data_df = pd.DataFrame(table_data_result, columns = cols)
  
  ## return the Table data as a DataFrame
  return table_data_df


In [8]:
# === TABLES of the DATABASE ===

# Department Table
department_data_df = getTableDataAsDataFrame(mycursor, 'department_data')
display(department_data_df.head());

,dept_id,dept_name,dept_head
0,D00-IT,IT,Henry Adey
1,D00-SS,Sales,Edward J Bayley
2,D00-TP,Temp,Micheal Zachrey
3,D00-ENG,Engineering,Sushant Raghunathan K
4,D00-SP,Support,Amelia Westray


In [9]:
# Employee Details table
employee_details_data_df = getTableDataAsDataFrame(mycursor, 'employee_details_data')
display(employee_details_data_df.head());

,employee_id,age,gender,marital_status
0,113558,43,Male,Married
1,112256,24,Female,Unmarried
2,112586,22,Female,Unmarried
3,108071,36,Male,Married
4,116915,38,Male,Married


In [10]:
# Employee data
### MAIN TABLE, Has most of the Features required for the model ###
employee_data_df = getTableDataAsDataFrame(mycursor, 'employee_data')
display(employee_data_df.head(10));


,avg_monthly_hrs,department,filed_complaint,last_evaluation,n_projects,recently_promoted,salary,satisfaction,status,tenure,employee_id
0,246.00,None,None,0.866838,6,NaN,medium,0.134415,Left,4.00,124467
1,134.00,None,None,0.555718,2,NaN,low,0.511041,Left,3.00,112210
2,156.00,D00-SS,1.00,0.474082,2,NaN,medium,0.405101,Left,3.00,126150
3,256.00,D00-SP,None,0.961360,6,NaN,low,0.152974,Left,4.00,125346
4,146.00,D00-SS,None,0.507349,2,NaN,medium,0.434845,Left,3.00,113707
5,135.00,D00-MT,None,0.482184,2,NaN,low,0.381545,Left,3.00,121685
6,270.00,D00-PD,None,0.867087,6,NaN,low,0.172575,Left,4.00,119912
7,244.00,D00-IT,None,0.926197,6,NaN,medium,0.061868,Left,5.00,112868
8,289.00,D00-IT,None,0.929858,7,NaN,low,0.161744,Left,4.00,122314
9,281.00,D00-SS,None,0.907965,6,NaN,medium,0.105749,Left,4.00,122394


In [12]:
employee_data_df.describe()

,last_evaluation,n_projects,recently_promoted,satisfaction,employee_id
count,12663.000000,14150.000000,297.0,14000.000000,14150.000000
mean,0.718399,3.778304,1.0,0.621212,112080.750247
std,0.173108,1.250162,0.0,0.250482,8748.202856
min,0.316175,1.000000,1.0,0.040058,0.000000
25%,0.563711,3.000000,1.0,0.450356,105772.500000
50%,0.724731,4.000000,1.0,0.652394,111291.500000
75%,0.871409,5.000000,1.0,0.824925,116650.750000
max,1.000000,7.000000,1.0,1.000000,148988.000000


In [ ]:
employee_data_df.describe()

In [22]:
employee_data_df['avg_monthly_hrs'].unique()

array([Decimal('246.00'), Decimal('134.00'), Decimal('156.00'),
       Decimal('256.00'), Decimal('146.00'), Decimal('135.00'),
       Decimal('270.00'), Decimal('244.00'), Decimal('289.00'),
       Decimal('281.00'), Decimal('269.00'), Decimal('267.00'),
       Decimal('257.00'), Decimal('155.00'), Decimal('128.00'),
       Decimal('274.00'), Decimal('151.00'), Decimal('127.00'),
       Decimal('132.00'), Decimal('309.00'), Decimal('130.00'),
       Decimal('233.00'), Decimal('245.00'), Decimal('149.00'),
       Decimal('232.00'), Decimal('284.00'), Decimal('249.00'),
       Decimal('164.00'), Decimal('159.00'), Decimal('154.00'),
       Decimal('239.00'), Decimal('260.00'), Decimal('125.00'),
       Decimal('308.00'), Decimal('306.00'), Decimal('141.00'),
       Decimal('143.00'), Decimal('261.00'), Decimal('301.00'),
       Decimal('296.00'), Decimal('271.00'), Decimal('129.00'),
       Decimal('290.00'), Decimal('225.00'), Decimal('253.00'),
       Decimal('255.00'), Decimal('268.0

In [13]:
employee_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14150 entries, 0 to 14149
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   avg_monthly_hrs    14150 non-null  object 
 1   department         13443 non-null  object 
 2   filed_complaint    2046 non-null   object 
 3   last_evaluation    12663 non-null  float64
 4   n_projects         14150 non-null  int64  
 5   recently_promoted  297 non-null    float64
 6   salary             14150 non-null  object 
 7   satisfaction       14000 non-null  float64
 8   status             14150 non-null  object 
 9   tenure             14000 non-null  object 
 10  employee_id        14150 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 1.2+ MB


In [21]:

## Get the Columns  of the table
mycursor.execute("show columns from employee_data")

cols_result = mycursor.fetchall()
cols_df = pd.DataFrame(cols_result)
cols_df.head(20)
#cols_df.iloc[:, 0]



,0,1,2,3,4,5
0,avg_monthly_hrs,"b'decimal(11,2)'",YES,[],None,
1,department,b'varchar(20)',YES,[],None,
2,filed_complaint,"b'decimal(11,2)'",YES,[],None,
3,last_evaluation,b'float',YES,[],None,
4,n_projects,b'int',YES,[],None,
5,recently_promoted,b'int',YES,[],None,
6,salary,b'varchar(20)',YES,[],None,
7,satisfaction,b'float',YES,[],None,
8,status,b'varchar(20)',YES,[],None,
9,tenure,"b'decimal(11,2)'",YES,[],None,


<a name = Section32></a>
### **3.2 Loading Test data**

---
<a name = Section4></a>
# **4. Pre-Profiling Report**
---

In [64]:
#!pip install pandas-profiling==2.7.1
#profile = ProfileReport(df = employee_train)
#profile.to_file(output_file = 'Pre Profiling Report.html')
#print('Accomplished!')

In [65]:
#from google.colab import files                   # Use only if you are using Google Colab, otherwise remove it
#files.download('Pre Profiling Report.html')      # Use only if you are using Google Colab, otherwise remove it

---
<a name = Section5></a>
# **5. Exploratory Data Analysis**
---

<a name = Section51></a>
### **5.1 Pairwise Plots**

<a name = Section52></a>
### **5.2 Heatmaps**

In [66]:
#### Detailed HEAT MAP of Correlations ####

---
<a name = Section6></a>
# **6. Feature Selection**
---

---
<a name = Section7></a>
# **7. Filling Missing / Null values if any**
---


<a name = Section71></a>
### **7.1 Data Description (Mean, median, std. dev. etc)**


<a name = Section72></a>
### **7.2 Fill missing / null values with Mean/Median/Mode**

---
<a name = Section8></a>
# **8. Feature Engineering**
---

---
<a name = Section9></a>
# **9. Data Preparation (before model creation)**
---

<a name = Section91></a>
### **9.1 Scaling**

<a name = Section92></a>
### **9.2 X and y creation**

<a name = Section93></a>
### **9.3 Train-Test Split**

<a name = Section94></a>
### **9.4 Scaling (Fit-Transform for Train, Transform for Test)**

---
<a name = Section10></a>
# **10. Model creation and prediction**
---

<a name = Section91></a>
### **10.1 Random Forest classification model**

<a name = Section102></a>
### **10.2 Prediction**

---
<a name = Section11></a>
# **11. Model Evaluation**
---